<a href="https://colab.research.google.com/github/fabrizioca2/ACS2024-1_GRUPO_06/blob/Fabrizio-Calixto/Algoritmo_GeneticoPreg3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install deap
!pip install control
import numpy as np
import control as ctrl
from deap import base, creator, tools, algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 9.3 MB/s eta 0:00:00


In [4]:
# Sistema de ejemplo: Planta G(s) = 1 / (s^2 + 3s + 2)
# Parámetros del sistema
M = 0.8  # Masa del carro (kg)
m = 0.23  # Masa del péndulo (kg)
l = 0.3  # Longitud del péndulo (m)
g = 9.81 # Aceleración de la gravedad (m/s²)

In [5]:
#Se define numerador y denominador
numerador = [1]
denominador = [M*l, 0, -(M+m)*g]

In [6]:
planta = ctrl.TransferFunction(numerador, denominador)

In [7]:
# Función objetivo: Evaluar rendimiento del controlador PID
def evaluar_pid(individuo):
    Kp, Ki, Kd = individuo
    # Controlador PID
    controlador = ctrl.TransferFunction([Kd, Kp, Ki], [1, 0])
    # Lazo cerrado
    sistema_lazo_cerrado = ctrl.feedback(controlador * planta)
    # Simulación de la respuesta al escalón
    t, yout = ctrl.step_response(sistema_lazo_cerrado)
    # Error: Deseamos que la salida siga un escalón unitario
    error = 1 - yout
    # ISE: Integral del error al cuadrado
    ise = np.sum(error**2) * (t[1] - t[0])
    return ise,

In [8]:
# Configuración del algoritmo genético
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar ISE
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

In [9]:
# Definir individuos (Kp, Ki, Kd)
toolbox.register("attr_float", np.random.uniform, 0, 10)  # Rango de [0, 10]
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [10]:
# Operadores genéticos
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluar_pid)

In [11]:
# Proceso evolutivo
population = toolbox.population(n=50)
ngen = 20  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruzamiento
mutpb = 0.2  # Probabilidad de mutación

In [12]:
# Se ejecuta algoritmo genético
resultados = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)
# Obtener los mejores parámetros PID
mejor_individuo = tools.selBest(population, k=1)[0]
print("Mejores ganancias PID encontradas:")
print(f"Kp = {mejor_individuo[0]}, Ki = {mejor_individuo[1]}, Kd = {mejor_individuo[2]}")

gen	nevals
0  	50    
1  	37    
2  	37    
3  	39    
4  	41    
5  	36    
6  	41    
7  	35    
8  	31    
9  	45    
10 	41    
11 	43    
12 	36    
13 	31    
14 	31    
15 	40    
16 	34    
17 	45    
18 	34    
19 	32    
20 	33    
Mejores ganancias PID encontradas:
Kp = 34.93141281628479, Ki = 26.29438807795288, Kd = 7.159550629812267
